In [87]:
import pandas as pd
import numpy as np
import unicodedata


import unicodedata

def remove_tildes(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )


In [88]:
def get_table(path, sheet_name, year):

    names_tables = ["cosecha", "produccion", "rendimiento", "precio"]

    # load sheet -------------------------------
    print("loading data ...")
    data = pd.read_excel(io=path, 
                        sheet_name=sheet_name)

    # recolled tables --------------------------------
    print("searching tables ....")
    list_tables = []
    found_table = False
    count = 0

    for row in data.itertuples():

        if found_table == True:
            count +=1
            if row[1] != "Nacional": # no include nacional row
                df.loc[len(df)] = row[1:]
                if row[1] == "Ucayali": # last row
                    list_tables.append(df)
                    found_table = False
                    count = 0      
        

        if row[1] == "Región":
            found_table = True
            df = pd.DataFrame(columns=row[1:])

    list_tables = list_tables[1:]
    print(len(list_tables), " tables found, ignoring the table of sembrada")

    ## metl tables

    print("melt tables")

    for i in range(len(list_tables)):
        df = list_tables[i].dropna(axis=1, how="all")
        df = df[['Región', 'Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul',
            'Ago', 'Set', 'Oct', 'Nov', 'Dic']]
        df = pd.melt(df , 
                id_vars="Región", 
                value_vars=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Set', 'Oct', 'Nov', 'Dic'],
                var_name="meses",
                value_name=names_tables[i]
            )
            
        #df['year'] = year
        list_tables[i] = df

    print("merge tables")
    df = list_tables[0]
    for table in list_tables[1:]:
        df = df.merge(table, on=["Región", "meses"])

    df["year"] = year
    
    return df


In [89]:
tables = get_table( path="./data/Cuadros en Excel del anuario _PRODUCCIÓN AGRÍCOLA_ 2019.xls", 
                    sheet_name='acelga',
                    year=2019)

loading data ...
searching tables ....
4  tables found, ignoring the table of sembrada
melt tables
merge tables


## excel, year, cultivo

In [90]:
cultivo = "Arroz"

paths = [
    '2730325-cuadros-en-excel-del-anuario-produccion-agricola-2022.xlsx',
    'Cuadros en Excel del anuario _PRODUCCIÓN AGRÍCOLA_ 2021.xls',
    'Cuadros en Excel del anuario _PRODUCCIÓN AGRÍCOLA_ 2020.xls',
    'Cuadros en Excel del anuario _PRODUCCIÓN AGRÍCOLA_ 2019.xls'
]

years = [2022, 2021, 2020, 2019]

tables = []

for year, table in zip(years, paths):
    tables.append(
        get_table(path="./data/"+table,
                sheet_name=cultivo,
                year=year))

data = pd.concat(tables, axis=0)
data.to_csv("produccion-cultivo-"+cultivo+".csv", index=False)


loading data ...
searching tables ....
4  tables found, ignoring the table of sembrada
melt tables
merge tables
loading data ...
searching tables ....
4  tables found, ignoring the table of sembrada
melt tables
merge tables
loading data ...
searching tables ....
4  tables found, ignoring the table of sembrada
melt tables
merge tables
loading data ...
searching tables ....
4  tables found, ignoring the table of sembrada
melt tables
merge tables


## ooo

In [91]:
cultivo = "cafe"

paths = [
    '2730325-cuadros-en-excel-del-anuario-produccion-agricola-2022.xlsx',
    'Cuadros en Excel del anuario _PRODUCCIÓN AGRÍCOLA_ 2021.xls',
    'Cuadros en Excel del anuario _PRODUCCIÓN AGRÍCOLA_ 2020.xls',
    'Cuadros en Excel del anuario _PRODUCCIÓN AGRÍCOLA_ 2019.xls'
]

years = [2022, 2021, 2020, 2019]

tables_month = []
tables_year = []

tables = {}

for year, table in zip(years, paths):
    
    data = pd.read_excel("./data/"+table, sheet_name=cultivo)
    starts = data.index[data.apply(lambda r: r.astype(str).str.contains("Cuadro").any(), axis=1)]
    print("Excel========================== ", year)
    for start in starts:
        
        for i in data.loc[start,:].dropna().values: # imprimiendo el titulo de la tabla
            if type(i)!=np.float64:
                print(remove_tildes(i.split(":")[1].split(",")[0]))
                name_table = remove_tildes(i.split(":")[1].split(",")[0].strip()).split(cultivo)[0] +" "+ cultivo
        
        
        df = data.loc[start+2:start+2+28,:].dropna(axis=1, how="all").reset_index(drop=True)
        if df.iloc[-1,0]!="Ucayali":
            df = df[:-1]
        if df.iloc[0,0]=="Región":
            df.columns = df.iloc[0,:]
            df = df.iloc[1:,:]
        df["year"] = year
        df = df[df["Región"].notna()]
        #print("dimension : ",df.shape)

        '''
        if df.shape[1] == 15:
            print("month ", df.shape)
            tables_month.append(df)
        if df.shape[1] == 6 :
            print("year ", df.shape)
            tables_year.append(df)
        '''
        if name_table not in tables:
            tables[name_table] = [df]
        else:
            tables[name_table].append(df)


   

Excel==========================  2022
 Produccion
 Produccion mensual de cafe pergamino
 Precio promedio en chacra mensual de cafe pergamino
Excel==========================  2021
 Produccion
 Produccion mensual de cafe
 Precio promedio en chacra mensual de cafe
Excel==========================  2020
 Produccion
 Produccion mensual de cafe
 Precio promedio en chacra mensual de cafe
Excel==========================  2019
 Produccion
 Produccion mensual de cafe
 Precio promedio en chacra mensual de cafe
 Superficie sembrada mensual de cafe
 Superficie cosechada mensual de cafe


In [92]:
list(tables.keys())

['Produccion cafe',
 'Produccion mensual de  cafe',
 'Precio promedio en chacra mensual de  cafe',
 'Superficie sembrada mensual de  cafe',
 'Superficie cosechada mensual de  cafe']

In [93]:
tables['Superficie cosechada mensual de  cafe'][0].head()

,Región,Total,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Set,Oct,Nov,Dic,year
1,Nacional,438177.097,0,0,0.75,13.5,72.25,32948,25913.588,221714.759,53357,61420.5,25827,16909.75,2019
2,Amazonas,54225,0,0,0,0,0,110,8259,9541,10640,7519,18110,46,2019
3,Áncash,0,0,0,0,0,0,0,0,0,0,0,0,0,2019
4,Apurímac,0,0,0,0,0,0,0,0,0,0,0,0,0,2019
5,Arequipa,0,0,0,0,0,0,0,0,0,0,0,0,0,2019


In [94]:
dim_check = True
head_check = True

for name_table in tables:

    dim = tables[name_table][0].shape
    head = tables[name_table][0].columns.values
    for dataframe in tables[name_table]:
        if dim == dataframe.shape and all(head==dataframe.columns.values):
            dim = dataframe.shape
            head = dataframe.columns.values
        
        else:
            print("dimention or head dont't macht in one dataframe")
            dim_check = False
            head_check = False
    
print("all frame correct dim, all head are correct")

all frame correct dim, all head are correct


In [95]:

if dim_check and head_check:
    for name_table in tables :
        tables[name_table] = pd.concat(tables[name_table])

In [96]:
tables['Superficie cosechada mensual de  cafe'].drop(columns=["Total"], inplace=True)

In [97]:
tables['Superficie cosechada mensual de  cafe'].tail()

,Región,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Set,Oct,Nov,Dic,year
23,Puno,0,0,0,0,0,0,0,10972,0,0,0,0,2019
24,San Martín,0,0,0,10,0,31843,13481,31111.1,5575,1921,75,9971,2019
25,Tacna,0,0,0,0,0,0,0,0,0,0,0,0,2019
26,Tumbes,0,0,0,0,0,0,0,0,0,0,0,0,2019
27,Ucayali,0,0,0,0,0,0,167.088,579.409,14435,0,0,0,2019


In [98]:
meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Set', 'Oct', 'Nov', 'Dic']
df =  tables['Superficie cosechada mensual de  cafe']
df = pd.melt(df , 
        id_vars= [col for col in df.columns if col not in meses],
        value_vars=meses,
        var_name="meses")
        
mes_map = {
    'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Ago': 8, 'Set': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
}
df['d-m-y'] = "1/"+ df['meses'].map(mes_map).astype(str) +"/"+ df.year.astype(str)
df.drop(columns=['year', "meses"], inplace=True)




In [99]:
df[df.Región=="Amazonas"]

,Región,value,d-m-y
1,Amazonas,0,1/1/2019
28,Amazonas,0,1/2/2019
55,Amazonas,0,1/3/2019
82,Amazonas,0,1/4/2019
109,Amazonas,0,1/5/2019
136,Amazonas,110,1/6/2019
163,Amazonas,8259,1/7/2019
190,Amazonas,9541,1/8/2019
217,Amazonas,10640,1/9/2019
244,Amazonas,7519,1/10/2019


In [100]:
"mensual" in "productos mensal"

False

In [101]:
for name_table in tables:
    if "mensual" in name_table:
        meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Set', 'Oct', 'Nov', 'Dic']
        df =  tables[name_table]
        df = pd.melt(df , 
                id_vars= [col for col in df.columns if col not in meses],
                value_vars=meses,
                var_name="meses")
                
        mes_map = {
            'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Ago': 8, 'Set': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
        }

        df['d-m-y'] = "1/"+ df['meses'].map(mes_map).astype(str) +"/"+ df.year.astype(str)
        df.drop(columns=['year', "meses"], inplace=True)

        tables[name_table] = df
        print("ola")


        

ola
ola
ola
ola


In [102]:
list(tables.keys())

['Produccion cafe',
 'Produccion mensual de  cafe',
 'Precio promedio en chacra mensual de  cafe',
 'Superficie sembrada mensual de  cafe',
 'Superficie cosechada mensual de  cafe']

In [103]:
df = tables['Superficie sembrada mensual de  cafe']
df[df.Región=="Amazonas"]

,Región,Total,value,d-m-y
1,Amazonas,72,2,1/1/2019
28,Amazonas,72,5,1/2/2019
55,Amazonas,72,0,1/3/2019
82,Amazonas,72,0,1/4/2019
109,Amazonas,72,0,1/5/2019
136,Amazonas,72,0,1/6/2019
163,Amazonas,72,0,1/7/2019
190,Amazonas,72,0,1/8/2019
217,Amazonas,72,0,1/9/2019
244,Amazonas,72,0,1/10/2019


In [104]:
[tables[name].shape for name in tables]

[(108, 6), (1296, 4), (1296, 4), (324, 4), (324, 3)]

In [108]:
tables['Superficie cosechada mensual de  cafe']

,Región,value,d-m-y
0,Nacional,0,1/1/2019
1,Amazonas,0,1/1/2019
2,Áncash,0,1/1/2019
3,Apurímac,0,1/1/2019
4,Arequipa,0,1/1/2019
...,...,...,...
319,Puno,0,1/12/2019
320,San Martín,9971,1/12/2019
321,Tacna,0,1/12/2019
322,Tumbes,0,1/12/2019


In [124]:
df = pd.read_csv("data.csv")

In [125]:
df

,Región,produccion,d-m-y,precio,sembrada,cosechada
0,Nacional,472.1,1/1/2019,5.034675,695,0.0
1,Amazonas,0.0,1/1/2019,0.000000,2,0.0
2,Áncash,0.0,1/1/2019,0.000000,0,0.0
3,Apurímac,0.0,1/1/2019,0.000000,0,0.0
4,Arequipa,0.0,1/1/2019,0.000000,0,0.0
...,...,...,...,...,...,...
319,Puno,0.0,1/12/2019,0.000000,0,0.0
320,San Martín,1693.0,1/12/2019,3.043119,228,9971.0
321,Tacna,0.0,1/12/2019,0.000000,0,0.0
322,Tumbes,0.0,1/12/2019,0.000000,0,0.0


In [19]:
sheet_names = [remove_tildes(name) for name in df.keys() if name not in ['Indice', 'siembras', 'cosecha', 'producción', 'rdto', 'precio', 'TRANSITORIOS']]

['acelga',
 'aji',
 'ajo',
 'albahaca',
 'apio',
 'arracacha',
 'Arroz',
 'arvejags',
 'arvejagv',
 'avenag',
 'betarraga',
 'brocoli',
 'caigua',
 'calabaza',
 'camote',
 'canihua',
 'cebada',
 'cebollacabeza',
 'cebollachina',
 'col',
 'coliflor',
 'espinaca',
 'fresa',
 'castilla',
 'Frijol',
 'frijolverd',
 'loctao',
 'garbanzo',
 'habags',
 'habagv',
 'kiwicha',
 'lechuga',
 'lenteja',
 'maca',
 'MAD',
 'Amilac',
 'choclo',
 'Mmorado',
 'mani',
 'mashua',
 'melon',
 'nabo',
 'oca',
 'olluco',
 'pallargs',
 'pallarverd',
 'Papa',
 'paprika',
 'pepinillo',
 'pepino',
 'pimiento',
 'piquillo',
 'pituca',
 'poro',
 'quinua',
 'rabanito',
 'sandia',
 'sorgo',
 'soya',
 'tarhui',
 'tomate',
 'Trigo',
 'vainita',
 'yuca',
 'zanahoria',
 'zapallo',
 'zarandaja',
 'PERMANENTES',
 'olivo',
 'achiot',
 'anona',
 'cacao',
 'cafe',
 'caimito',
 'camucamu',
 'capuli',
 'carambola',
 'cereza',
 'chirimoya',
 'cirolero',
 'ciruela',
 'coco',
 'cocona',
 'damasco',
 'datil',
 'granada',
 'guanaban